# FF5 Stock Factors

In [8]:
import dai
import pandas as pd

In [9]:
sd = '2023-01-01'
ed = '2024-01-01'

## 1. FF5 Stock Factors SQL

In [10]:
def get_ff5_sql(sql_base):
    sql = f"""--sql
    WITH
    data1 AS (
        {sql_base}
    ),

    data_mkt AS (
        SELECT DISTINCT
            date,
            c_sum(float_market_cap * change_ratio) / c_sum(float_market_cap) AS MKT
        FROM data1
    ),

    hv AS (
        WITH hv0 AS (
            SELECT
                date,
                instrument,
                change_ratio,
                float_market_cap,
                c_pct_rank(float_market_cap) AS rank_sb,
                c_pct_rank(bp_ratio)         AS rank_lmh,
                CASE WHEN rank_sb < 0.5 THEN 1 ELSE 2 END AS group_sb,
                CASE
                    WHEN rank_lmh < 0.3 THEN 1
                    WHEN rank_lmh > 0.7 THEN 3
                    ELSE 2
                END AS group_lmh
            FROM data1
        ),
        SL AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SL
            FROM hv0 WHERE group_sb=1 AND group_lmh=1
        ),
        SM_v AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SM_v
            FROM hv0 WHERE group_sb=1 AND group_lmh=2
        ),
        SH AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SH
            FROM hv0 WHERE group_sb=1 AND group_lmh=3
        ),
        BL AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BL
            FROM hv0 WHERE group_sb=2 AND group_lmh=1
        ),
        BM_v AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BM_v
            FROM hv0 WHERE group_sb=2 AND group_lmh=2
        ),
        BH AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BH
            FROM hv0 WHERE group_sb=2 AND group_lmh=3
        )
        SELECT
            SL.date AS date,
            (1/3)*(SL + SM_v + SH) - (1/3)*(BL + BM_v + BH) AS SMB_HML,
            (1/2)*(SH + BH)        - (1/2)*(SL + BL)        AS HML
        FROM SL
        JOIN SM_v USING(date)
        JOIN SH   USING(date)
        JOIN BL   USING(date)
        JOIN BM_v USING(date)
        JOIN BH   USING(date)
    ),
    hp AS (
        WITH hp0 AS (
            SELECT
                date,
                instrument,
                change_ratio,
                float_market_cap,
                c_pct_rank(float_market_cap) AS rank_sb,
                c_pct_rank(prof_ratio)       AS rank_wmr,
                CASE WHEN rank_sb < 0.5 THEN 1 ELSE 2 END AS group_sb,
                CASE
                    WHEN rank_wmr < 0.3 THEN 1
                    WHEN rank_wmr > 0.7 THEN 3
                    ELSE 2
                END AS group_wmr
            FROM data1
            QUALIFY prof_ratio IS NOT NULL
        ),
        SW AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SW
            FROM hp0 WHERE group_sb=1 AND group_wmr=1
        ),
        SM_p AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SM_p
            FROM hp0 WHERE group_sb=1 AND group_wmr=2
        ),
        SR AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SR
            FROM hp0 WHERE group_sb=1 AND group_wmr=3
        ),
        BW AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BW
            FROM hp0 WHERE group_sb=2 AND group_wmr=1
        ),
        BM_p AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BM_p
            FROM hp0 WHERE group_sb=2 AND group_wmr=2
        ),
        BR AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BR
            FROM hp0 WHERE group_sb=2 AND group_wmr=3
        )
        SELECT
            SW.date AS date,
            (1/3)*(SW + SM_p + SR) - (1/3)*(BW + BM_p + BR) AS SMB_RMW,
            (1/2)*(SR + BR)        - (1/2)*(SW + BW)        AS RMW
        FROM SW
        JOIN SM_p USING(date)
        JOIN SR   USING(date)
        JOIN BW   USING(date)
        JOIN BM_p USING(date)
        JOIN BR   USING(date)
    ),
    hi AS (
        WITH hi0 AS (
            SELECT
                date,
                instrument,
                change_ratio,
                float_market_cap,
                c_pct_rank(float_market_cap) AS rank_sb,
                c_pct_rank(inv_ratio)        AS rank_cma,
                CASE WHEN rank_sb < 0.5 THEN 1 ELSE 2 END AS group_sb,
                CASE
                    WHEN rank_cma < 0.3 THEN 1
                    WHEN rank_cma > 0.7 THEN 3
                    ELSE 2
                END AS group_cma
            FROM data1
            QUALIFY inv_ratio IS NOT NULL
        ),
        SC AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SC
            FROM hi0 WHERE group_sb=1 AND group_cma=1
        ),
        SM_i AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SM_i
            FROM hi0 WHERE group_sb=1 AND group_cma=2
        ),
        SA AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SA
            FROM hi0 WHERE group_sb=1 AND group_cma=3
        ),
        BC AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BC
            FROM hi0 WHERE group_sb=2 AND group_cma=1
        ),
        BM_i AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BM_i
            FROM hi0 WHERE group_sb=2 AND group_cma=2
        ),
        BA AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BA
            FROM hi0 WHERE group_sb=2 AND group_cma=3
        )
        SELECT
            SC.date AS date,
            (1/3)*(SC + SM_i + SA) - (1/3)*(BC + BM_i + BA) AS SMB_CMA,
            (1/2)*(SC + BC)        - (1/2)*(SA + BA)        AS CMA
        FROM SC
        JOIN SM_i USING(date)
        JOIN SA   USING(date)
        JOIN BC   USING(date)
        JOIN BM_i USING(date)
        JOIN BA   USING(date)
    ),

    data_ff5 AS (
        SELECT
            m.date,
            m.MKT,
            (hv.SMB_HML + hp.SMB_RMW + hi.SMB_CMA) / 3 AS SMB,
            hv.HML,
            hp.RMW,
            hi.CMA
        FROM data_mkt m
        JOIN hv USING(date)
        JOIN hp USING(date)
        JOIN hi USING(date)
        QUALIFY COLUMNS(*) IS NOT NULL
    )
    SELECT *
    FROM data_ff5
    ORDER BY date
    """
    return sql


## 2. FF5 Factors

### 2.1 FF5 Daily

In [11]:
sql_ff5_base_daily = f"""--sql
SELECT
    date,
    instrument,
    change_ratio,
    float_market_cap,
    1 / pb                     AS bp_ratio,
    net_profit_rate_lf         AS prof_ratio,
    invested_capital_lf_yoy    AS inv_ratio
FROM cn_stock_prefactors
WHERE instrument NOT LIKE '%BJ%'
"""

sql_ff5_daily = get_ff5_sql(sql_ff5_base_daily)

In [12]:
df_ff5_daily = dai.query(sql_ff5_daily, filters={"date":[sd, ed]}).df()
df_ff5_daily

,date,MKT,SMB,HML,RMW,CMA
0,2023-01-03,0.010998,0.015268,-0.004603,-0.003603,0.002195
1,2023-01-04,0.002176,0.004357,0.008139,-0.003562,0.007957
2,2023-01-05,0.013000,-0.008526,-0.014943,0.009229,-0.013476
3,2023-01-06,0.001170,-0.003763,-0.006305,0.008799,-0.010890
4,2023-01-09,0.006535,-0.000008,-0.004306,0.000809,-0.000520
...,...,...,...,...,...,...
237,2023-12-25,0.002003,-0.008841,-0.005550,0.003849,-0.004892
238,2023-12-26,-0.008222,-0.005391,0.005306,0.002695,-0.001564
239,2023-12-27,0.005632,0.002380,-0.001444,-0.000445,-0.000001
240,2023-12-28,0.017699,0.001538,-0.008186,0.000559,-0.009065


### 2.2 FF5 Weekly

In [13]:
sql_ff5_base_weekly = """
WITH 
data_orgn AS (
    SELECT
        date,
        instrument,
        close,
        float_market_cap,
        pb,
        net_profit_rate_lf,
        invested_capital_lf_yoy
    FROM cn_stock_prefactors
),

data_freq AS (
    WITH 
    data_temp_1 AS (
        SELECT
            instrument,
            CONCAT(CAST(year(date) AS CHAR), CAST(week(date) AS CHAR)) AS period,

            FIRST_VALUE(date) OVER (
                PARTITION BY period, instrument 
                ORDER BY date 
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
            ) AS date,

            LAST_VALUE(close) OVER (
                PARTITION BY period, instrument 
                ORDER BY date 
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
            ) AS close,

            LAST_VALUE(float_market_cap) OVER (
                PARTITION BY period, instrument 
                ORDER BY date 
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
            ) AS float_market_cap,

            LAST_VALUE(pb) OVER (
                PARTITION BY period, instrument 
                ORDER BY date 
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
            ) AS pb,

            LAST_VALUE(net_profit_rate_lf) OVER (
                PARTITION BY period, instrument 
                ORDER BY date 
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
            ) AS prof_ratio,

            LAST_VALUE(invested_capital_lf_yoy) OVER (
                PARTITION BY period, instrument 
                ORDER BY date 
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
            ) AS inv_ratio,

            ROW_NUMBER() OVER (PARTITION BY period, instrument ORDER BY date DESC) AS rn
        FROM data_orgn
    ),

    data_temp_2 AS (
        SELECT
            date,
            instrument,
            close,
            LAG(close) OVER (PARTITION BY instrument ORDER BY period) AS close_w_lag_1,
            float_market_cap,
            pb,
            prof_ratio,
            inv_ratio
        FROM data_temp_1
        WHERE rn = 1
    )

    SELECT
        date,
        instrument,
        close / close_w_lag_1 - 1 AS change_ratio,
        float_market_cap,
        1 / pb AS bp_ratio,
        prof_ratio,
        inv_ratio
    FROM data_temp_2
)

SELECT
    date,
    instrument,
    change_ratio,
    float_market_cap,
    bp_ratio,
    prof_ratio,
    inv_ratio
FROM data_freq
WHERE instrument NOT LIKE '%BJ%'
ORDER BY date, instrument
"""

sql_ff5_weekly = get_ff5_sql(sql_ff5_base_weekly)

In [14]:
df_ff5_weekly = dai.query(sql_ff5_weekly, filters={"date":[sd, ed]}).df()
df_ff5_weekly

,date,MKT,SMB,HML,RMW,CMA
0,2023-01-09,0.035558,-0.019914,-0.155243,0.028188,-0.085519
1,2023-01-16,0.079507,-0.044459,-0.200385,0.037787,-0.041075
2,2023-01-30,0.200125,-0.144454,-0.271747,0.081416,-0.186072
3,2023-02-06,0.192790,-0.129297,-0.248397,0.047756,-0.156018
4,2023-02-13,-0.012301,-0.004072,0.004109,-0.001559,0.008417
5,2023-02-20,0.011990,-0.002575,0.009568,0.001360,0.003584
6,2023-02-27,0.015465,-0.012781,0.013716,-0.011732,0.017328
7,2023-03-06,0.035022,0.028906,-0.010232,-0.022253,0.037287
8,2023-03-13,0.002497,-0.019434,0.012719,-0.010587,0.015878
9,2023-03-20,0.017927,0.001601,-0.021531,-0.005032,-0.001781


### 2.3 FF5 Monthly

In [15]:
sql_ff5_base_monthly = """
WITH 
data_orgn AS (
    SELECT
        date,
        instrument,
        close,
        float_market_cap,
        pb,
        net_profit_rate_lf,
        invested_capital_lf_yoy
    FROM cn_stock_prefactors
),
data_freq AS (
    WITH 
    data1 AS (
        SELECT
            instrument,
            CONCAT(CAST(year(date) AS CHAR), CAST(month(date) AS CHAR)) AS period,

            FIRST_VALUE(date) OVER (
                PARTITION BY period, instrument 
                ORDER BY date 
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
            ) AS date,

            LAST_VALUE(close) OVER (
                PARTITION BY period, instrument 
                ORDER BY date 
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
            ) AS close,

            LAST_VALUE(float_market_cap) OVER (
                PARTITION BY period, instrument 
                ORDER BY date 
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
            ) AS float_market_cap,

            LAST_VALUE(pb) OVER (
                PARTITION BY period, instrument 
                ORDER BY date 
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
            ) AS pb,

            LAST_VALUE(net_profit_rate_lf) OVER (
                PARTITION BY period, instrument 
                ORDER BY date 
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
            ) AS prof_ratio,

            LAST_VALUE(invested_capital_lf_yoy) OVER (
                PARTITION BY period, instrument 
                ORDER BY date 
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
            ) AS inv_ratio,

            ROW_NUMBER() OVER (PARTITION BY period, instrument ORDER BY date DESC) AS rn
        FROM data_orgn
    ),
    data2 AS (
        SELECT
            date,
            instrument,
            close,
            LAG(close) OVER (PARTITION BY instrument ORDER BY period) AS close_m_lag_1,
            float_market_cap,
            pb,
            prof_ratio,
            inv_ratio
        FROM data1
        WHERE rn = 1 
    )
    SELECT 
        date,
        instrument,
        close / close_m_lag_1 - 1 AS change_ratio,
        float_market_cap,
        1 / pb AS bp_ratio,
        prof_ratio,
        inv_ratio
    FROM data2
)
SELECT
    date,
    instrument,
    change_ratio,
    float_market_cap,
    bp_ratio,
    prof_ratio,
    inv_ratio
FROM data_freq
WHERE instrument NOT LIKE '%BJ%'
ORDER BY date, instrument
"""

sql_ff5_monthly = get_ff5_sql(sql_ff5_base_monthly)

In [16]:
df_ff5_monthly = dai.query(sql_ff5_monthly, filters={"date":[sd, ed]}).df()
df_ff5_monthly

,date,MKT,SMB,HML,RMW,CMA
0,2023-02-01,0.184054,-0.130650,-0.215686,0.043323,-0.133270
1,2023-03-01,0.010762,-0.037960,-0.029034,-0.019719,0.023423
2,2023-04-03,0.012244,-0.053866,0.023377,0.048321,0.010725
3,2023-05-04,-0.023972,0.037204,-0.021567,0.002464,0.013088
4,2023-06-01,0.022841,0.008400,-0.038238,-0.003430,0.000175
5,2023-07-03,0.040959,-0.041853,0.040181,-0.010019,0.030162
6,2023-08-01,-0.049022,0.020751,-0.035505,0.019495,-0.000654
7,2023-09-01,-0.002098,-0.000203,0.012825,-0.009967,0.017288
8,2023-10-09,-0.007133,-0.020292,-0.027054,0.024634,0.055271
9,2023-11-01,0.009180,0.039600,-0.031618,-0.018660,0.019966


## 3. FF5 Analysis

In [17]:
def ff5_analysis(df, n):
    
    import pandas as pd
    from sklearn.linear_model import LinearRegression

    results = []

    for instrument, group in df.groupby('instrument'):

        group = group.sort_values('date')

        beta_ICP  = []
        beta_MKT  = []
        beta_SMB  = []
        beta_HML  = []
        beta_RMW  = []
        beta_CMA  = []
        alpha     = []
        regr_resd = []
        regr_pred = []
        regr_r2   = []

        if len(group) < n:
            group['beta_ICP']  = [None] * len(group)
            group['beta_MKT']  = [None] * len(group)
            group['beta_SMB']  = [None] * len(group)
            group['beta_HML']  = [None] * len(group)
            group['beta_RMW']  = [None] * len(group)
            group['beta_CMA']  = [None] * len(group)
            group['alpha']     = [None] * len(group)
            group['regr_resd'] = [None] * len(group)
            group['regr_pred'] = [None] * len(group)
            group['regr_r2']   = [None] * len(group)

        else:
            for i in range(len(group) - n + 1):

                window = group.iloc[i:i+n]
                y = window['Return'].values
                X = window[['MKT', 'SMB', 'HML', 'RMW', 'CMA']].values

                model = LinearRegression()
                model.fit(X, y)

                yhat_last = model.predict(X)[-1]

                beta_ICP.append(model.intercept_)
                beta_MKT.append(model.coef_[0])
                beta_SMB.append(model.coef_[1])
                beta_HML.append(model.coef_[2])
                beta_RMW.append(model.coef_[3])
                beta_CMA.append(model.coef_[4])

                regr_pred.append(yhat_last)
                regr_resd.append(y[-1] - yhat_last)
                regr_r2.append(model.score(X, y))

                # 按你原来的定义：alpha = 截距 + 最后一条残差
                alpha.append(model.intercept_ + (y[-1] - yhat_last))

            pad = [None] * (n - 1)
            group['beta_ICP']  = pad + beta_ICP
            group['beta_MKT']  = pad + beta_MKT
            group['beta_SMB']  = pad + beta_SMB
            group['beta_HML']  = pad + beta_HML
            group['beta_RMW']  = pad + beta_RMW
            group['beta_CMA']  = pad + beta_CMA
            group['alpha']     = pad + alpha
            group['regr_resd'] = pad + regr_resd
            group['regr_pred'] = pad + regr_pred
            group['regr_r2']   = pad + regr_r2

        results.append(group)

    result_df = pd.concat(results).sort_index()
    return result_df


In [18]:
df_ff5_daily_cs = pd.merge(dai.query("""
SELECT
    date,
    instrument,
    change_ratio AS Return
FROM cn_stock_prefactors
""", filters = {"date":[sd,ed]}).df(), df_ff5_daily, how = "inner", on = ["date"])

In [19]:
df_ff5_derived_daily = ff5_analysis(df_ff5_daily_cs, 20)
df_ff5_derived_daily

,date,instrument,Return,MKT,SMB,HML,RMW,CMA,beta_ICP,beta_MKT,beta_SMB,beta_HML,beta_RMW,beta_CMA,alpha,regr_resd,regr_pred,regr_r2
0,2023-01-03,000001.SZ,0.046353,0.010998,0.015268,-0.004603,-0.003603,0.002195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-03,000002.SZ,0.001648,0.010998,0.015268,-0.004603,-0.003603,0.002195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-03,000004.SZ,0.026860,0.010998,0.015268,-0.004603,-0.003603,0.002195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-03,000005.SZ,0.005376,0.010998,0.015268,-0.004603,-0.003603,0.002195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-03,000006.SZ,-0.028571,0.010998,0.015268,-0.004603,-0.003603,0.002195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260979,2023-02-14,603855.SH,-0.005078,0.001545,-0.001280,0.004398,0.000877,-0.001089,-0.001694,0.830714,2.597920,1.098653,3.327090,2.367758,-0.008206,-0.006512,1.433714e-03,0.208307
1260980,2023-02-14,603856.SH,0.014528,0.001545,-0.001280,0.004398,0.000877,-0.001089,0.000423,0.847824,1.372661,0.024423,3.781575,3.121577,0.014951,0.014528,-6.098681e-07,0.379385
1260981,2023-02-14,603858.SH,0.015144,0.001545,-0.001280,0.004398,0.000877,-0.001089,-0.003472,0.001299,-0.364142,-2.376331,2.869350,4.496174,0.027505,0.030977,-1.583313e-02,0.294431
1260982,2023-02-14,603859.SH,-0.016104,0.001545,-0.001280,0.004398,0.000877,-0.001089,0.010178,0.175385,-1.977370,-2.139773,-0.598067,0.752599,-0.008153,-0.018331,2.227146e-03,0.148456
